In [ ]:
#| default_exp docstring

# Docstring helpers

In [ ]:
#| export

from typing import *
import sys
import os

import re

import textwrap
from subprocess import run, CalledProcessError  # nosec: B404
from tempfile import TemporaryDirectory
from pathlib import Path

from rich import print
from rich.console import Group, Console
from rich.panel import Panel

In [ ]:
import pytest
import numpy as np

In [ ]:
def f(i: str, *, a: int = 0):
    """Very cool function

    **f** is a very cool function

    Args:
        i: something
        a: something else

    Example:
        The following snippet prints out greetings for two names:
        ```python
        print("hello {fill in name_1}")
        print("goodbye {fill in name_2}")
        ```


    Example:
        Yet another example
        ```python
        password = {fill in password}
        print(f"this is your password: {password}")

        
        from  nbdev_mkdocs.docstring import run_examples_from_docstring

        print("Logging in...")
        ```

    """
    raise NotImplemented()

In [ ]:
#| export


def _extract_examples_from_docstring(o: Any) -> List[str]:
    try:
        import griffe
    except:
        raise Exception(
            "This function should only be used for testing where griffe package is installed."
        )

    if o.__doc__ is None:
        raise ValueError(f"{o.__name__}.__doc__ = {o.__doc__}")
    sections = griffe.docstrings.parse(
        griffe.dataclasses.Docstring(o.__doc__), griffe.docstrings.Parser.google
    )
    
    def find_python_code(s: str) -> str:
        code = [x[0] for x in re.findall("```\s*python((\n|.|\\n])+)```", s)]
        #         print(f"{code=}")
        if len(code) == 0:
            raise ValueError(f"No python code found in {s}")
        return "\n\n".join(code)

    examples = [
        find_python_code(section.value.description)  # type: ignore
        for section in sections
        if section.kind.value == "admonition" and section.value.annotation == "example"  # type: ignore
    ]

    return examples

In [ ]:
def g():
    """Function

    Example:
        ```python
        from airt.helpers import print_header

        print("hello world")
        ```
        
    Example:
        ``` python
        from nbdev_mkdocs.package_data import get_root_data_path

        req_path = get_root_data_path() / "requirements.txt"
        print(f"Path is: {req_path.resolve()}")
        assert req_path.exists()
        ```
    """


_extract_examples_from_docstring(g)

['\nfrom airt.helpers import print_header\n\nprint("hello world")\n',
 '\nfrom nbdev_mkdocs.package_data import get_root_data_path\n\nreq_path = get_root_data_path() / "requirements.txt"\nprint(f"Path is: {req_path.resolve()}")\nassert req_path.exists()\n']

In [ ]:
_extract_examples_from_docstring(f)

<module>:5: No type or annotation for parameter 'i'
<module>:6: No type or annotation for parameter 'a'


['\nprint("hello {fill in name_1}")\nprint("goodbye {fill in name_2}")\n',
 '\npassword = {fill in password}\nprint(f"this is your password: {password}")\n\n\nfrom  nbdev_mkdocs.docstring import run_examples_from_docstring\n\nprint("Logging in...")\n']

In [ ]:
expected = [
    textwrap.dedent(
        """
            print("hello {fill in name_1}")
            print("goodbye {fill in name_2}")
        """
    ),
    textwrap.dedent(
        """
            password = {fill in password}
            print(f"this is your password: {password}")


            from  nbdev_mkdocs.docstring import run_examples_from_docstring
            
            print("Logging in...")
        """
    ),
]

examples = _extract_examples_from_docstring(f)
for example in examples:
    print(example)

np.testing.assert_array_equal(examples, expected)

<module>:5: No type or annotation for parameter 'i'
<module>:6: No type or annotation for parameter 'a'


print("hello {fill in name_1}")
print("goodbye {fill in name_2}")

password = {fill in password}
print(f"this is your password: {password}")


from  nbdev_mkdocs.docstring import run_examples_from_docstring

print("Logging in...")

In [ ]:
examples[0] == expected[0]

True

In [ ]:
#| export


def _get_keywords(examples: List[str]) -> List[str]:
    keywords: List[str] = sum(
        [
            [x[9:-1] for x in re.findall("{fill in \w+}", example)]
            for example in examples
        ],
        start=[],
    )

    return keywords

In [ ]:
expected = ["name_1", "name_2", "password"]

keywords = _get_keywords(examples)

np.testing.assert_array_equal(keywords, expected)

In [ ]:
#| export


def _replace_keywords(examples: List[str], **kwargs) -> List[str]:
    keywords = _get_keywords(examples)

    if set(keywords) > set(kwargs.keys()):
        raise ValueError(f"{set(keywords)} > {set(kwargs.keys())}")

    for keyword in keywords:
        examples = [
            example.replace("{fill in " + keyword + "}", kwargs[keyword])
            for example in examples
        ]

    return examples

In [ ]:
expected = [
    textwrap.dedent(
        """
            print("hello davor")
            print("goodbye kumaran")
        """
    ),
    textwrap.dedent(
        """
            password = 'not_a_password'
            print(f"this is your password: {password}")
            

            from  nbdev_mkdocs.docstring import run_examples_from_docstring

            print("Logging in...")
        """
    ),
]

actual = _replace_keywords(
    examples, name_1="davor", name_2="kumaran", password="'not_a_password'"
)
for x in actual:
    print(x)

np.testing.assert_array_equal(actual, expected)

print("hello davor")
print("goodbye kumaran")

password = 'not_a_password'
print(f"this is your password: {password}")


from  nbdev_mkdocs.docstring import run_examples_from_docstring

print("Logging in...")

In [ ]:
#| export


def _format_output(
    s: str,
    *,
    title: str,
    supress: bool = False,
    sub_dict: Optional[Dict[str, str]] = None,
):
    if sub_dict:
        for pattern, replacement in sub_dict.items():
            s = re.sub(pattern, replacement, s)
    if supress:
        return Panel("", title=f"{title} supressed")
    else:
        return Panel(s, title=title)

In [ ]:
Panel(
    Group(
        "",
        _format_output("hello world and one more time world", title="output"),
        _format_output(
            "hello world and one more time world",
            title="output",
            sub_dict={"world": "*****"},
        ),
        _format_output(
            "hello world and one more time world",
            title="output",
            sub_dict={"world": "*****"},
            supress=True,
        ),
    ),
    title="Test: _format_output()"
)

╭──────────────────────────────────────────── Test: _format_output() ─────────────────────────────────────────────╮
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── output ───────────────────────────────────────────────────╮ │
│ │ hello world and one more time world                                                                         │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── output ───────────────────────────────────────────────────╮ │
│ │ hello ***** and one more time *****                                                                         │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── output supressed ──────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
#| export


def run_examples_from_docstring(
    o: Any,
    *,
    supress_stdout: bool = False,
    supress_stderr: bool = False,
    sub_dict: Optional[Dict[str, str]] = None,
    **kwargs
):
    """Runs example from a docstring

    Parses docstring of an objects looking for examples. The examples are then saved into files and executed
    in a separate process.

    Note:
        Execution context is not the same as the one in the notebook because we want examples to work from
        user code. Make sure you compiled the library prior to executing the examples, otherwise you might
        be running them agains an old version of the library.

    Args:
        o: an object, typically a function or a class, for which docstring is being parsed for examples
        supress_stdout: omit stdout from output, typically due to security considerations
        supress_stderr: omit stderr from output, typically due to security considerations
        sub_dict: a dictionary mapping regexp patterns into replacement strings used to mask stdout and
            stderr, typically used to mask sensitive information such as passwords

        **kwargs: arguments use to replace "{fill in **param**}" in docstring with the actual values when running examples

    Raises:
        ValueError: if some params are missing from the **kwargs**
        RuntimeException: if example fails

    Example:
        ```python
        from  nbdev_mkdocs.docstring import run_examples_from_docstring

        def f():
            ```python
            Example:
                print("Hello {fill in name}!")
                print("Goodbye {fill in other_name}!")
            ```
            pass


        run_examples_from_docstring(f, name="John", other_name="Jane")
        ```
    """
    examples = _extract_examples_from_docstring(o)
    if len(examples) == 0:
        raise ValueError(f"No examples found in:\n{o.__doc__}")
        
    executable_examples = _replace_keywords(examples, **kwargs)
    for example, executable_example in zip(examples, executable_examples):
        with TemporaryDirectory() as d:
            cmd_path = (Path(d) / "example.py").absolute()
            with open(cmd_path, "w") as f:
                f.write(executable_example)
            process = run(  # nosec: B603
                [sys.executable, str(cmd_path)], capture_output=True, text=True
            )
            panel_group = Group(
                "Example:\n" + textwrap.indent(example, " " * 4),
                _format_output(process.stdout, title="stdout", supress=supress_stdout, sub_dict=sub_dict),
                _format_output(process.stderr, title="stderr", supress=supress_stderr, sub_dict=sub_dict),
            )
            print(Panel(panel_group))
            if process.returncode != 0:
                raise RuntimeError(process.stderr)

In [ ]:
print(run_examples_from_docstring.__doc__)

Runs example from a docstring

    Parses docstring of an objects looking for examples. The examples are then saved into files and executed
    in a separate process.

    Note:
        Execution context is not the same as the one in the notebook because we want examples to work from
        user code. Make sure you compiled the library prior to executing the examples, otherwise you might
        be running them agains an old version of the library.

    Args:
        o: an object, typically a function or a class, for which docstring is being parsed for examples
        supress_stdout: omit stdout from output, typically due to security considerations
        supress_stderr: omit stderr from output, typically due to security considerations
        sub_dict: a dictionary mapping regexp patterns into replacement strings used to mask stdout and
            stderr, typically used to mask sensitive information such as passwords

        **kwargs: arguments use to replace "{fill in **param**}" in docstring with the actual values when running 
examples

    Raises:
        ValueError: if some params are missing from the **kwargs**
        RuntimeException: if example fails

    Example:
        ```python
        from  nbdev_mkdocs.docstring import run_examples_from_docstring

        def f():
            ```python
            Example:
                print("Hello {fill in name}!")
                print("Goodbye {fill in other_name}!")
            ```
            pass


        run_examples_from_docstring(f, name="John", other_name="Jane")
        ```

In [ ]:
from nbdev_mkdocs.docstring import run_examples_from_docstring

examples = _extract_examples_from_docstring(run_examples_from_docstring)
console = Console(width=64)
console.print(Panel(examples[0], width=64))

<module>:11: No type or annotation for parameter 'o'
<module>:12: No type or annotation for parameter 'supress_stdout'
<module>:13: No type or annotation for parameter 'supress_stderr'
<module>:14: No type or annotation for parameter 'sub_dict'
<module>:17: No type or annotation for parameter '**kwargs'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭──────────────────────────────────────────────────────────────╮
│                                                              │
│ from  nbdev_mkdocs.docstring import                          │
│ run_examples_from_docstring                                  │
│                                                              │
│ def f():                                                     │
│     ```python                                                │
│     Example:                                                 │
│         print("Hello {fill in name}!")                       │
│         print("Goodbye {fill in other_name}!")               │
│     ```                                                      │
│     pass                                                     │
│                                                              │
│                                                              │
│ run_examples_from_docstring(f, name="John",                  │
│ other_name="Jane")                                           │
│                                                              │
╰──────────────────────────────────────────────────────────────╯

In [ ]:
run_examples_from_docstring(
    f, name_1="davor", name_2="all", password='"zeko"', supress_stderr=True, sub_dict={"zeko": "*"*12}
)

<module>:5: No type or annotation for parameter 'i'
<module>:6: No type or annotation for parameter 'a'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     print("hello {fill in name_1}")                                                                             │
│     print("goodbye {fill in name_2}")                                                                           │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ hello davor                                                                                                 │ │
│ │ goodbye all                                                                                                 │ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── stderr supressed ──────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     password = {fill in password}                                                                               │
│     print(f"this is your password: {password}")                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│     from  nbdev_mkdocs.docstring import run_examples_from_docstring                                             │
│                                                                                                                 │
│     print("Logging in...")                                                                                      │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ this is your password: ************                                                                         │ │
│ │ Logging in...                                                                                               │ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── stderr supressed ──────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
with pytest.raises(ValueError) as e:
    run_examples_from_docstring(f, name_1="davor")

e.value

<module>:5: No type or annotation for parameter 'i'
<module>:6: No type or annotation for parameter 'a'


ValueError("{'password', 'name_2', 'name_1'} > {'name_1'}")

In [ ]:
class C:
    """Cool class with broken example

    Example:
        ```python
        from nbdev_mkdocs.docstring import run_examples_from_docstring
        
        raise NotImplementedError("expected to fail")
        ```
    """

    pass

In [ ]:
with pytest.raises(RuntimeError) as e:
    run_examples_from_docstring(C)
e.value

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     from nbdev_mkdocs.docstring import run_examples_from_docstring                                              │
│                                                                                                                 │
│     raise NotImplementedError("expected to fail")                                                               │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── stderr ───────────────────────────────────────────────────╮ │
│ │ Traceback (most recent call last):                                                                          │ │
│ │   File "/tmp/tmppia530fd/example.py", line 4, in <module>                                                   │ │
│ │     raise NotImplementedError("expected to fail")                                                           │ │
│ │ NotImplementedError: expected to fail                                                                       │ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

RuntimeError('Traceback (most recent call last):\n  File "/tmp/tmppia530fd/example.py", line 4, in <module>\n    raise NotImplementedError("expected to fail")\nNotImplementedError: expected to fail\n')

In [ ]:
from nbdev_mkdocs.package_data import get_root_data_path

print(get_root_data_path.__doc__)

Returns path to package_data in lib

    Example:
        ``` python
        from nbdev_mkdocs.package_data import get_root_data_path

        req_path = get_root_data_path() / "requirements.txt"
        print(f"Path is: {req_path.resolve()}")
        assert req_path.exists()
        
        ```

In [ ]:
_extract_examples_from_docstring(get_root_data_path)

['\nfrom nbdev_mkdocs.package_data import get_root_data_path\n\nreq_path = get_root_data_path() / "requirements.txt"\nprint(f"Path is: {req_path.resolve()}")\nassert req_path.exists()\n\n']